In [ ]:
from google.colab import drive

drive.mount('/drive')

Mounted at /drive


In [ ]:
#!git clone https://github.com/jwkirchenbauer/lm-watermarking
!pip install -q transformers accelerate

fatal: destination path 'lm-watermarking' already exists and is not an empty directory.


In [ ]:
cd '/drive/My Drive/Colab Notebooks/ISO/'

/drive/.shortcut-targets-by-id/1k2JFo63MNzskIgAHpOe33d-BrKTiK7mE/ISO


In [ ]:
cd kirchenbauer-lm-watermarking

/drive/.shortcut-targets-by-id/1k2JFo63MNzskIgAHpOe33d-BrKTiK7mE/ISO/kirchenbauer-lm-watermarking


In [ ]:
!pip install -q -r new_requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.3/533.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17

In [ ]:
import os
import argparse
import pandas as pd
from argparse import Namespace
from pprint import pprint
from functools import partial

import numpy # for gradio hot reload

import torch
from tqdm import tqdm

from transformers import (AutoTokenizer,
                          AutoModelForSeq2SeqLM,
                          AutoModelForCausalLM,
                          LogitsProcessorList)

from extended_watermark_processor import WatermarkLogitsProcessor, WatermarkDetector

In [ ]:
checkpoint = 'facebook/opt-1.3b'

model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
watermark_processor = WatermarkLogitsProcessor(vocab=list(tokenizer.get_vocab().values()),
                                               gamma=0.25,
                                               delta=2.0,
                                               seeding_scheme="algorithm-3") #equivalent to `ff-anchored_minhash_prf-4-True-15485863`
# Note:
# You can turn off self-hashing by setting the seeding scheme to `minhash`.

In [ ]:
cd '../instances'

/drive/.shortcut-targets-by-id/1k2JFo63MNzskIgAHpOe33d-BrKTiK7mE/ISO/instances


In [ ]:
import string
p = set(string.printable)

In [ ]:
batch_size = 8
filename = 'kirchenbauer_no_attack.json'

df = pd.read_json('c4_selection.json', lines=True)

with open(filename, "r+" if os.path.exists(filename) else "w+") as f:
    with torch.no_grad():
        for i in tqdm(range(len(f.readlines()), len(df["input"]), batch_size)):
            line = df["input"][i:i+batch_size].values.tolist()
            tokenized_input = tokenizer(line, return_tensors='pt').to(model.device)
            # note that if the model is on cuda, then the input is on cuda
            # and thus the watermarking rng is cuda-based.
            # This is a different generator than the cpu-based rng in pytorch!

            output_tokens = model.generate(**tokenized_input, max_new_tokens=250,
                                            logits_processor=LogitsProcessorList([watermark_processor]))

            # if decoder only model, then we need to isolate the
            # newly generated tokens as only those are watermarked, the input/prompt is not

            for j in range(len(output_tokens)):
                output_text = tokenizer.decode(output_tokens[j][51:], skip_special_tokens=True)

                _line = ''.join(filter(lambda x: x in p, line[j])).replace('\n', '\\n').replace('"', '\\"')
                _output_text = ''.join(filter(lambda x: x in p, output_text)).replace('\n', '\\n').replace('"', '\\"')

                f.write(f'{{"input": "{_line}", "continuation": "{_output_text}"}}\n')

100%|██████████| 7/7 [35:33<00:00, 304.73s/it]
